## Capture images

In [ ]:
import time
import cv2
import datetime as dt
from timelapse.utils import calculate_brightness, find_correct_exposure, get_properties
from notify_run import Notify
notify = Notify()

In [ ]:
starttime = time.time()
notification_sent = False
exposure = 100

while True:
    try:
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)
        cap.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap,exposure, n_iter = find_correct_exposure(cap,100,125,1000)
        timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        ret,frame = cap.read()
        props = get_properties(cap)
        cv2.imwrite(f'images/capture_{timestamp}.png',frame)
        cap.release()
        cv2.destroyAllWindows()
        
        with open("log.txt", "a+") as f:
            f.write(f'{timestamp} --- \nexposure: {exposure}, brightness: {calculate_brightness(frame):.2f}\n')
            f.write(f'props: {props}\n\n')
        
    except:
        if not notification_sent:
            notify.send('Timelapse has run into an error')
            notification_sent=True
        try:
            cap.release()
            cv2.destroyAllWindows()
        except:
            pass

        
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))